# Strange Biomes
A Cyberthon 2021 project for image classification of Minecraft biome images.

In [75]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from torchvision.io import read_image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [76]:
train_df = pd.read_csv('train.csv')
train_df

,image,biome_name,shore,tundra,warm,shattered,bamboo,lukewarm,desert,river,...,flower,eroded,tree,ocean,ice,jungle,frozen,edge,gravelly,taiga
0,a4b578cad321bbb97d3dc529daa632bce3685cb00500e9...,badlands,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2c2d7cd3bdbee03263256ff78a52184efa8f51c34947d5...,badlands,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,f3c19617f29bdb228157e5648340e89e19bf2b84624475...,badlands,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,53214611fce6f9dc96355bec658a07111a2df84bd9457b...,badlands,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,b40619a2c66022b97746a410b456f208cc4dc65a7d24ee...,badlands,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26182,3934060fdd8a1e9fe7db3992c6d036f62850855563a29e...,wooded_mountains,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26183,a3c994270d2619d3e710a4f43a2d3465fd004ebb8bf8b5...,wooded_mountains,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26184,c7381b1a9a1e4610cfff4f70df9f24971526d4357dc6e3...,wooded_mountains,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26185,8b241326d9c02c7c16efc4055599d80bd8187b3b298b98...,wooded_mountains,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
class dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        sample = {"image": image, "label": label}
        return sample

In [78]:
data_dir = '/Users/kieranmendoza/PycharmProjects/nn-strange-biomes/train-smaller'

def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(28),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(28),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader
train_dataloader, test_dataloader = load_split_train_test(data_dir, .2)
print(test_dataloader.dataset.classes)

['badlands', 'badlands_plateau', 'bamboo_jungle', 'bamboo_jungle_hills', 'beach', 'birch_forest', 'birch_forest_hills', 'cold_ocean', 'dark_forest', 'dark_forest_hills', 'deep_cold_ocean', 'deep_frozen_ocean', 'deep_lukewarm_ocean', 'deep_ocean', 'desert', 'desert_hills', 'desert_lakes', 'eroded_badlands', 'flower_forest', 'forest', 'frozen_ocean', 'frozen_river', 'giant_tree_taiga', 'giant_tree_taiga_hills', 'gravelly_mountains', 'ice_spikes', 'jungle', 'jungle_edge', 'jungle_hills', 'lukewarm_ocean', 'modified_gravelly_mountains', 'modified_jungle', 'modified_wooded_badlands_plateau', 'mountains', 'mushroom_field_shore', 'mushroom_fields', 'ocean', 'plains', 'river', 'savanna', 'savanna_plateau', 'shattered_savanna', 'shattered_savanna_plateau', 'snowy_beach', 'snowy_mountains', 'snowy_taiga', 'snowy_taiga_hills', 'snowy_taiga_mountains', 'snowy_tundra', 'stone_shore', 'sunflower_plains', 'swamp', 'swamp_hills', 'taiga', 'taiga_hills', 'taiga_mountains', 'tall_birch_forest', 'tall_bi

In [79]:
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 3, 28, 49])
Shape of y:  torch.Size([64]) torch.int64


In [80]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [81]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = Net()
#model = models.resnet50(pretrained=True)
print(model)

Using cpu device
Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=13456, out_features=64, bias=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [82]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
model.to(device)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=13456, out_features=64, bias=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [83]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [84]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [85]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [8, 3, 5, 5], but got 2-dimensional input of size [64, 4116] instead

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")